# Astro 528, Lab 4, Exercise 1

## Optimization problems & Optimizing Code:  Type Stability & Profiling

A common problem is to find the optimum (i.e., either minimum or maximum) value of a function.  While this can be straightforward for some well-behaved problems, for others it is extremely challenging.  There are numerous algorithms depending on the details of the properties of the function to be optimized (e.g., continuous vs discrete, number of dimensions), the availability of additional information (e.g., gradients) or constraints.  This exercise will introduce you to a few of the common optimization packages available in Julia.  In the process, you'll get some experience learn about profiling and optimizing a serial function.  

Be aware that the field of optimization is extremely rich and there are numerous algorithms, libraries and packages, most specializing in some more specific class of optimization problems.  For a list of Julia packages for optimization, see [JuliaOpt's packages page](http://www.juliaopt.org/packages/) and the [JuliaNLSolvers organization](https://github.com/JuliaNLSolvers).

In [ ]:
# First Install any packages needed
using Pkg
Pkg.activate(".")
Pkg.instantiate()

## Fitting a model to data

A common task in Astronomy & Astrophysics is to fit a model to data, varying the model parameters to find the "best-fit" values.  This is a form of optimization problem.  Often one maximizes goodness-of-fit criterion such as the log-likelihood (or minimize a loss function, such as $\chi^2$).  In this exercise, we'll consider a very simple model, create a simulated dataset and (attempt to) find the best-fit model parameters. 

First, you'll follow along with the demonstrations trying to anticipate what will happen and comparing your expectations to reality.  

## Linear optimization

Consider the case of using linear regression to find the coefficients of a polynomial model, $y(x) = a_0 + a_1 \times x + a_2 \times x^2$.  For example, $x$ might be the column of a detector and $y$ might be the wavelength of light being accumulated at that column.  A simple implementation is provided in `wavelength_model(x,a)`.  Note that it takes an optional parameter `center`, which could be beneficial for reducing correlations between coefficients of different terms.

In [ ]:
"Compute the wavelength as a function of pixel (x) using polynomial model (coeff)."
function wavelength_model(x, coeff::Array; center = zero(typeof(x)) )
    @assert 2 <= length(coeff) <= 10
    y = coeff[1]                       # constant
    for i in 1:(length(coeff)-1)
        y += coeff[i+1] * (x-center)^i
    end
    return y
end

In order to measure the coefficients, we might make $N$ calibration measurements denoted by $\hat{y_i} = y_i + \epsilon_i = a_0 + a_1 \times x_i + a_2 \times x_i^2 + \epsilon_i$.  Here $\epsilon_i \sim N(0,\sigma_i^2) $ would be the measurement error for the $i$th wavelgnth measurement and is assumed to be iid normally distributed about zero with variance $\sigma_i^2$.  In this case, the maximum likelihood solution is equivalent to least squares regression.  Therefore, we can find the best fit values, $\vec{a}_{bf} = \mathrm{argmin}_{\vec{a}} \sum_{i=1}^{N_{obs}} \left| \frac{y_i - y(x_i)}{\sigma_i} \right|^2$, via  linear algebra.

To demonstrate, let's start by creating a simulated dataset.

In [ ]:
# Fixed parameters for our model
N_obs = 100               # number of measurements
N_pixels = 1024           # number of pixels on detector
polynomial_degree = 3     # degree of polynomial to use
x_center = floor(Float64,N_pixels//2)  # center polynomial, so as to reduce correlations between coefficients

# Generate simulated data
poly_coeff_true = [5000,1,1e-5, 1e-9] 
sigma_y = 0.1             # measurement uncertainties
# Pick a set of columns where we'll make measurements
x_obs = convert(Array{Float64,1},sort(rand(1:N_pixels, N_obs)))
# Simulation observations using true model, adding measurement noise
y_obs = map(x->wavelength_model(x, poly_coeff_true,center=x_center) + sigma_y * randn(), x_obs)

In [ ]:
"Compute best-fit coefficients for wavelength model using linear algebra for linear least squares regression"
function fit_wavelength_model_linear_alg(x_obs, y_obs; degree::Integer=1, x_center = zero(eltype(x_obs)) )
   @assert 1 <= degree < 10
   @assert length(x_obs) == length(y_obs) >= degree+1
    
   design_matrix = Array{eltype(x_obs)}(undef,length(x_obs),degree+1)
   design_matrix[:,1] .= 1.0
   for i in 1:(degree)   
     design_matrix[:,i+1] .= (x_obs.-x_center).^i
   end
   coef_fit = design_matrix \ y_obs
end

What values do you expect the result will be close to?

In [ ]:
result_linalg = fit_wavelength_model_linear_alg(x_obs, y_obs, degree=polynomial_degree, x_center=x_center)

First, it's always wise to double check that your functions are returning reasonable results.  Compare the output best-fit parameters to the true model parameters (`poly_coeff_true`) used to generate the data.  If they aren't very similar, then we should fix that before thinking about performance.

Now, let's benchmark the code.

In [ ]:
using BenchmarkTools

In [ ]:
@btime fit_wavelength_model_linear_alg(x_obs, y_obs, degree=polynomial_degree, x_center=x_center)

Second, wow, that was fast!  
That speed was possible because this problem is a form of linear optimization.  Here _linear_ refers to the predictions of the model being linear function of the coefficients for a fixed value of x (and not that the model is linear in x).  For linear optimization (without constraints), there is a single global best-fit solution and it can be found quickly using linear algebra.  

Note the time required and the amount of memory allocated for a given $N_{obs}$.  

### Least Squares Regression via Levenberg-Marquardt
Often the predictions of physical models are non-linear functions of the model parameters.  In these cases, we'll need to move from linear optimization to _non-linear optimization_.  This can be much more challenging and computationally intensive.  First, it's not obvious that there will be one global optimum.  If there are just one or two parameters, then it may be practical to perform a thorough exploration of parameter space to look for each local optimum.  However, this often becomes impractical in higher dimensions.  Therefore, we often settle for using _local optimization_ algorithms whose result depends on the initialization.  If we have a good idea of where the optimum is likely to be, then we might provide an initial guess.  Otherwise, we might try running a local optimization algorithm several times using different initial guesses to increase the chances that we find the global optimum.  
  
Fitting a non-linear model to data is such a common operation that there's a package, [LsqFit.jl](https://github.com/JuliaNLSolvers/LsqFit.jl), for this sort of problem.  LsqFit.jl uses the Levenberg-Marquardt algoritihm to try to find the values of the model parameters that minimize the $\chi^2 = \sum_i^{N_{obs}} \left| \frac{y_i - y(x_i)}{\sigma_i} \right|^2$.  

For pedagogical purposes, we'll continue using the same linear model as above, even as we try some algorithms capable of working with non-linear models.  In this example, we'll assume that a good initial guess is available.

In [ ]:
using LsqFit

In [ ]:
function wavelength_model_centered_v1(x_obs, coeff)
    wavelength_model_setting_x_center(x) = wavelength_model(x,coeff,center=x_center) 
    wavelength_model_setting_x_center.(x_obs)
end
param_guess = [4500,0.8,1e-6,1e-11]
weights = fill(1/sigma_y^2,length(x_obs));

In [ ]:
result_lsqfit = curve_fit(wavelength_model_centered_v1, x_obs, y_obs, weights, param_guess)
result_lsqfit.param

If we were to test for equality of the coefficients from the two algorithms, they'd be slightly different.  Even allowing for floating point issues, the fractional error in the cubic coefficient could easily cause a test to fail (e.g., if used default values of `isapprox`.

In [ ]:
maximum(abs.((result_linalg .- result_lsqfit.param) ./result_linalg))

In this case, what we're really interested in is how the predictions of the models differ. 
Therefore, it's useful to build a test around the predictions.

In [ ]:
maximum(abs.((wavelength_model_centered_v1(x_obs,result_linalg) .- wavelength_model_centered_v1(x_obs,result_lsqfit.param))./wavelength_model_centered_v1(x_obs,result_linalg) ))

Hopefully, the predictions of the model identified by the non-linear optimization algorithm are very similar to the predictions of the linear optimization.  


Next, let's look at the runtime and memory allocations used by `curve_fit`.

In [ ]:
@btime curve_fit(wavelength_model_centered_v1, x_obs, y_obs, weights, param_guess)

How does the run-time compare to linear optimiziation for the same model and number of observations?


INSERT RESPONSE

Some of the difference is very likely due to the fact that Levenberg-Marquardt is an iterative algorithm that does not assume the model is linear.  However, it's always good to check if there could be some inefficiency in your code.  

Perhaps the easiest way to write Julia code that's not as efficient as it could be is by writing code that is not _type stable_.  That is, the compiler can't be certain what type the variables will have at runtime, thus preventing it from compiling efficient code.  Look at `wavelength_model_centered_v1`.  Do you see anything that could prevent this function from being type stable?

INSERT RESPONSE

Often, it's hard to spot type instability by eye.  Fortunately, there are a few tools to help us identify when this happens.  For example, there's the `@code_warntype` macro. It shows you what Julia can deduce about the type of every line of your code.  For starters, let's try our simplest function, `wavelength_model`.

In [ ]:
@code_warntype wavelength_model(1.0,poly_coeff_true,center=x_center)

Wow, that's a lot of info.  The key thing for us is that each line ends in an annotation of what type it returns and it's always a single type (shown in a pleasant light blue for my settings).  Now, let's try that for `wavelength_model_centered_v1`.

In [ ]:
@code_warntype wavelength_model_centered_v1(x_obs, poly_coeff_true)

Wow, that's even more info.  The key thing to notice is that towards the end, some of the return types are annotated as `Any` (or a `Union{..}`, `DataType`, etc.).  That means that this function is _not_ type-safe, i.e., the compiler can't deduce the type of the return value of some of lines of code.  To improve performance We'd like to fix that.  You're welcome to try to make sense of all the output above, but there is an easier way.  Fortunately, someone has written a package ([`Traceur`](https://github.com/JunoLab/Traceur.jl)) that analyzes the output of `@code_warntype` and provides a more readable description of the most common issues.  Let's try it, first on the type-safe function and then on the type-unsafe function.

In [ ]:
using Traceur

In [ ]:
@trace wavelength_model(1.0,poly_coeff_true,center=x_center);

In the case above, the `@trace` macro didn't find any issues, so it kept quiet.  How, let's try it on our type-unsafe function.

In [ ]:
@trace wavelength_model_centered_v1(x_obs, poly_coeff_true);

The first warning message contains the key clue to the origin of issue.  We've referred to x_center, a global variable.  Even though `x_center` is currently a `Float64`, that could change, and the compiler has to allow for that in several places below.  Finally, it warns that the return type of `wavelength_model_centered` is `Any`.  This means that code that calls `wavelength_model_centered` will also be type-unstable.  

There are multiple ways we could solve this.  We could change the function to take x_center as an optional parameter (like `wavelength_model`).  We could create a composite type that contains both the polynomial coefficients and `x_center` with a fixed (or parameterized) type.  For simplicity, I'll suggest simply creating a local variable equal to `x_center`, but providing a _type annotation_ that makes a promise to the compiler that the type of x_center will always be a `Float64`.  If that's not true, then it will return an error.

In [ ]:
function wavelength_model_centered_v2(x_obs, coeff)
    xc = x_center::Float64
    wavelength_model_setting_x_center(x) = wavelength_model(x,coeff,center=xc) 
    wavelength_model_setting_x_center.(x_obs)
end
@trace wavelength_model_centered_v2(x_obs, poly_coeff_true);

Excellent, `@trace` founds no type-instability.  With that simple change, our function is now type stable.  

Now let's try benchmarking the Levenberg-Marquardt algorithm using a type-stable version of the model function.

In [ ]:
result_lsqfit_v2 = @btime curve_fit(wavelength_model_centered_v2, x_obs, y_obs, weights, param_guess).param

How does the runtime compare to using the same algorithm with the type-unstable function?  

How does the runtime compare to that of the linear optimizer?  

INSERT RESPONCE

In theory, you could have applied the `@trace` macro to the `curve_fit` function.  In practice, that would likely be annoyingly slow, because it would be analyzing so much code.  You can tell @trace that you only want it to look one or two function calls deep by providing an optional , `maxdepth` parameter.  For example,

In [ ]:
@trace( curve_fit(wavelength_model_centered_v2, x_obs, y_obs, weights, param_guess), maxdepth=3)

## Profiling

Now that we've eliminated the unnecessary inefficiencies in our function due to things like global variables and type instability, it is a good time to profile our code to see where it's spending its time.  For this, we'll use Julia's `Profile` module.

In [ ]:
using Profile

In [ ]:
Profile.init(n = 10^6, delay = 0.00001)  # Tell profiler how frequently to check what line of code is running
#Profile.clear()               # Remember to clear() profiler history if you run @profile multiple times
@profile result_lsqfit = curve_fit(wavelength_model_centered_v2, x_obs, y_obs, weights, param_guess)

Now, that we've profiled the code, it's time to look at the results.

In [ ]:
Profile.print()

The output is quite detailed.  You could scroll through every line, looking for which functions that we wrote are taking a large fraction of the time.  However, often, it's better to start looking at a graphical representation of the profiler results.  For that, we'll use the [`ProfileView.jl`](https://github.com/timholy/ProfileView.jl) pacakge

In [ ]:
PROFILEVIEW_USEGTK = true   # true to make flame plot appear as a separate window
using ProfileView

In [ ]:
ProfileView.closeall()      # Prevent confusion from two windows 
ProfileView.view();

Hopefully, The bottom lines show the outer level function calls.  If each row isn't already labled, then hover your mouse over a bar to see what function and line number it represents.  Because we're working interactively, some rows at the bottom that aren't particularly relevant.  Starting at the bottom, shift your attention upwards until you reach the bar labeled "macro expansion in Profile.jl...".  This bar represents the entire time that was profiled.  Most of the time was spent calling other functions, as can be seen be observing the bars in the row above are nearly as wide as the bar labeled "Profile.jl...".  These functions call other functions and at the very top you can see the small functions being called.  For plain .jl files, it's simply the line number.  For code in a Jupyter notebook, the cell number is in square brackets and the line within that cell after the semicolon.  (ProfileView also tries to be helpful by hiding some of the smallest functions like accessing an element of an array or copying data, since most programmers are very unlikely to improve the efficiency of those.)  

Look at the second-to-top row.  What function is taking the most time?

INSERT RESPONCE

Within that function, what is taking most of the time?  

INSERT RESPONCE

The good news is that most of the time is doing some real calculations.  However, if we inspect the third-from top row, then we may be in for a suprise.  In this case, most of its time is being spent inside a function with `finite_difference` in its name.  Whether we realized it or not, the `curve_fit` function was using finite differences to estimate derivatives of the model predictions.  Realizing that's where it's spending most of our time, we can recognize that we might help the Levenberg-Marquardt algorithm to calculate derivatives more efficiently (and more accurately) by providing a Jacobian matrix.  That is, we'll give it a function that can calculate how the derivative of each prediction depends on the independent variable and model parameters.  Let's try that.

In [ ]:
function jacobian_wavelength_model_v1(x, coeff; center = 0)
    @assert length(x) >= length(coeff)
    @assert length(coeff) >= 2 

    J = Array{Float64}(undef, length(x), length(coeff))
    J[:,1] .= 1.0
    for i in 2:length(coeff)
        J[:,i] .= (x.-center).^(i-1)
    end
    return J
end
function jacobian_wavelength_model_centered_v1(x,coeff) 
    xc = x_center::Float64
    jacobian_wavelength_model_v1(x,coeff,center=xc)
end

First, let's double check that our new function is still type-stable.

In [ ]:
@trace(jacobian_wavelength_model_centered_v1(x_obs, poly_coeff_true), maxdepth=3);

Ok, now what do you expect the run-time will be for curve_fit when we provide the Jacobian, so it doesn't need to calculate derivatives analytically?

In [ ]:
result_lsqfit_with_jacobian = @btime curve_fit(wavelength_model_centered_v2, 
    jacobian_wavelength_model_centered_v1, x_obs, y_obs, weights, param_guess)
result_lsqfit_with_jacobian.param

First double check that the code found very nearly the same optimum as before.  Once, you're convinced the code is correct, then let's think about performance?  

How did the run time of the Levenberg-Marquardt algorithm using a Jacobian compare to the linear optimization algorithm?

INSERT RESPONCE

Let's profile our new code.

In [ ]:
#Profile.init(n = 10^6, delay = 0.00001)  # Tell profiler how frequently to check what line of code is running
Profile.clear()     # Only need to clear() if you run @profile multiple times
@profile curve_fit(wavelength_model_centered_v2, jacobian_wavelength_model_centered_v1, 
                            x_obs, y_obs, weights, param_guess)

In [ ]:
ProfileView.closeall()
ProfileView.view();

If you wanted to further improve the performance of optimizing this model, then you could try eliminating the calls to the `^` operator.  Since we're only using a cubic polynomial, it's likely write our create model as
$a_0 + x * (a_1 + x * ( a_2 + x * a_3 )$.  Let's try that...

In [ ]:
wavelength_model_old = wavelength_model

"Compute the wavelength as a function of pixel (x) using polynomial model (coeff)."
function wavelength_model(x, coeff::Array; center = zero(typeof(x)) )
    @assert 2 <= length(coeff) <= 10
    y = coeff[end]                       
    for i in (length(coeff)-1):-1:1      # a range starting at degree and running backwards to 1
        y *= (x-center)
        y += coeff[i]
    end
    return y
end

@assert wavelength_model(123,poly_coeff_true,center=x_center) ≈ wavelength_model_old(123,poly_coeff_true,center=x_center)

In [ ]:
result_lsqfit_with_jacobian = @btime curve_fit(wavelength_model_centered_v2, 
            jacobian_wavelength_model_centered_v1, x_obs, y_obs, weights, param_guess)
result_lsqfit_with_jacobian.param

How does the runtime compare to the previous version?  

INSERT RESPONSE

Let's profile to code again.

In [ ]:
Profile.init(delay = 0.00001)  # Tell profiler how frequently to check what line of code is running
Profile.clear()     # Only need to clear() if you run @profile multiple times
@profile result_lsqfit = curve_fit(wavelength_model_centered_v2, jacobian_wavelength_model_centered_v1, x_obs, y_obs, weights, param_guess)
ProfileView.closeall()
ProfileView.view();

If you wanted to improve the performance further, what might you try next?
    
INSERT RESPONSE

In [ ]:
ProfileView.closeall()

## More General Optimization 

For some problems, we want to optimize a function that is more complex than simply fitting a model to data.  Perhaps, we're including penalty or regularlization terms or trying to find the mode of a posterior probability density.  Or maybe we just want to test whether a different algorithm might be better suited to our problem.  
For example, we could use the more flexible [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package that has multiple optimization algorithms that are able to optimize continuous functions that are more general than just $\chi^2$.  Therefore, we must explicitly specify what function we would like it to minimize.  Below, we'll just compute $\chi^2$, so we can make fair comparisons.  There are different choices of algorithms that make use of zero, one or two derivatives of the objective function.

In [ ]:
using Optim

In [ ]:
function chi_sq_wavelength_model(coef)
    sum(((wavelength_model_centered_v2(x_obs,coef) .- y_obs )./sigma_y).^2)
end

What's inefficient about the above function?  Revise the function `chi_sq_wavelength_model` so that it is more efficient.
    
INSERT RESPONSE

In [ ]:
# INSERT SOLUTION

We'll start with the default algorithm (Nelder-Mead) that doesn't use any derivatives.

In [ ]:
result_optim_neldermead = @btime optimize(chi_sq_wavelength_model,param_guess)

How did the runtime of `optimize` using its default Nelder-Mead algorithm compare to the runtime of LsqFit's `curve_fit` using Levenberg-Marquardt?

INSERT RESPONSE

Next, we'll try two algorithms that make use of the gradient of our target function, [Conjugate Gradient Descent](https://en.wikipedia.org/wiki/Conjugate_gradient_method) and the 
[BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) algorithm (or [L-BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS) that uses less memory).  

First, let's try Conjugate Gradient Descent.  How do you expect it's performance to compare to that of the Nelder-Mead algorithm?

INSERT RESPONSE

In [ ]:
result_optim_conjgrad = @btime optimize(chi_sq_wavelength_model,param_guess,ConjugateGradient())

Rather than us having to compute the derivatives of the target function manually, we can tell the `optimize` function to compute the derivatives using _automatic differentiation_.  Instead of having to estimate gradients using finite differences, it will compute them analytically.  Before you run the next cell, what do you expect for the performance of the Conjugate Gradient Descent optimizer using automatic differentiation?

INSERT RESPONSE

In [ ]:
result_optim_conjgrad_autograd = @btime optimize(chi_sq_wavelength_model,param_guess,ConjugateGradient(),autodiff = :forward)

How did the runtime compare to that of Conjugate Gradient Descent without using automatic differentiation?  How did the number of evaluations of the objective function (i.e., the function we're minimizing) and the number of evaluations of the gradient compare?  Why was this one faster?

INSERT RESPONSE

How did the performance compare to the Levenberg-Marquardt?

INSERT RESPONSE

For some problems, the _BFGS_ algorithm can be significantly more efficient.  Let's try it... first without using automatic differentiation.

In [ ]:
result_optim_bfgs = @btime optimize(chi_sq_wavelength_model,param_guess,BFGS())

Now let's try using the BFGS algorithm with automatic differentiation.

In [ ]:
result_optim_bfgs_autodiff = @btime optimize(chi_sq_wavelength_model,param_guess,BFGS(),autodiff = :forward)

How did the performance of the BFGS algorithm compare to the Conjugate Gradient Descent?  How did the number of objective and gradient call compare?  

INSERT RESPONSE

When choosing an optimization algorithm, what properties would you want to consider to maximize the performance?


The LsqFit and Optim packages often work well for fairly small models.  For students interested in performing optimization over large models, you'll likely want to look into algorithms such as stochastic gradient descent and/or using GPUs.  For these, I'd suggest that you check out the [Flux.jl](https://github.com/FluxML/Flux.jl) package.  It's designed for machine learning, but can also be useful for generic large optimization problems.